<a href="https://colab.research.google.com/github/toexploremind/liuer/blob/my_new_branch01/Lecture_12_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Sep 12 02:06:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch

In [3]:
batch_size = 1
seq_len = 3   #即x1,x2,x3
input_size = 4 #即x的长度
hidden_size = 2 

cell = torch.nn.RNNCell(input_size=input_size, hidden_size=hidden_size)

dataset = torch.randn(seq_len, batch_size, input_size)
hidden = torch.zeros(batch_size, hidden_size)

for idx, input in enumerate(dataset): #依次输入x1,x2,x3的batch
  print('=' * 20, idx, '=' *20)
  print('Input size:', input.shape)

  hidden = cell(input, hidden)

  print('hidden size:', hidden.shape)
  print(hidden)
        

==================== 0 ====================
Input size: torch.Size([1, 4])
hidden size: torch.Size([1, 2])
tensor([[0.8712, 0.7014]], grad_fn=<TanhBackward0>)
==================== 1 ====================
Input size: torch.Size([1, 4])
hidden size: torch.Size([1, 2])
tensor([[0.6974, 0.9949]], grad_fn=<TanhBackward0>)
==================== 2 ====================
Input size: torch.Size([1, 4])
hidden size: torch.Size([1, 2])
tensor([[0.3346, 0.9136]], grad_fn=<TanhBackward0>)


主要是要弄清输入和隐层的维度

In [4]:
import torch

batch_size = 1
seq_len = 3   #即x1,x2,x3
input_size = 4 #即x的长度
hidden_size = 2 
num_layers = 1

cell = torch.nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers = num_layers)

inputs = torch.randn(seq_len, batch_size, input_size)
hidden = torch.zeros(num_layers, batch_size, hidden_size)

out, hidden = cell(inputs, hidden)

print('Oouput size', out.shape)
print('Output:', out)
print('Hidden size:', hidden.shape)
print('Hidden:', hidden)

Oouput size torch.Size([3, 1, 2])
Output: tensor([[[-0.0856,  0.4983]],

        [[ 0.7732,  0.6507]],

        [[-0.3539,  0.1777]]], grad_fn=<StackBackward0>)
Hidden size: torch.Size([1, 1, 2])
Hidden: tensor([[[-0.3539,  0.1777]]], grad_fn=<StackBackward0>)


In [ ]:
import torch

input_size = 4
hidden_size = 4
batch_size = 1

idx2char = ['e', 'h', 'l', 'o']
x_data = [1, 0, 2, 2, 3]
y_data = [3, 1, 2, 3, 2]

one_hot_lookup = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]

x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = torch.Tensor(x_one_hot).view(-1, batch_size, input_size)
labels = torch.LongTensor(y_data).view(-1, 1)

In [5]:
class Model(torch.nn.Module):
  def __init__(self, input_size, hidden_size, batch_size):
    super(Model, self).__init__()
    self.batch_size = batch_size
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.rnncell = torch.nn.RNNCell(input_size = self.input_size, hidden_size = self.hidden_size)

  def forward(self, input, hidden):
    hidden = self.rnncell(input, hidden)
    return hidden

  def init_hidden(self):
    return torch.zeros(self.batch_size, self.hidden_size)
  
net = Model(input_size, hidden_size, batch_size)


In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.1)

In [ ]:
for epoch in range(15):
  loss = 0
  optimizer.zero_grad()
  hidden = net.init_hidden()
  print('Predicted string:', end='')
  for input,label in zip(inputs, labels):
    hidden = net(input, hidden)
    loss += criterion(hidden,label)  
    _, idx = hidden.max(dim=1)
    print(idx2char[idx.item()], end='')
  loss.backward()
  optimizer.step()
  print(', Epoch [%d/15] loss=%.4f' % (epoch+1, loss.item()))


Predicted string:hohhh, Epoch [1/15] loss=7.2502
Predicted string:hoool, Epoch [2/15] loss=6.0265
Predicted string:ooool, Epoch [3/15] loss=5.2078
Predicted string:ooool, Epoch [4/15] loss=4.7264
Predicted string:ooool, Epoch [5/15] loss=4.4053
Predicted string:ooool, Epoch [6/15] loss=4.1423
Predicted string:ohool, Epoch [7/15] loss=3.9166
Predicted string:ohool, Epoch [8/15] loss=3.6938
Predicted string:ohool, Epoch [9/15] loss=3.4427
Predicted string:ohool, Epoch [10/15] loss=3.1624
Predicted string:ohool, Epoch [11/15] loss=2.8865
Predicted string:ohool, Epoch [12/15] loss=2.6752
Predicted string:ohool, Epoch [13/15] loss=2.5461
Predicted string:ohool, Epoch [14/15] loss=2.4638
Predicted string:ohlol, Epoch [15/15] loss=2.4024
